In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render ul li{font-size:22pt; line-height:30px;}
div.output {font-size:22pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:22pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:22pt;padding:5px;}
table.dataframe{font-size:22px;}
</style>
"""))

<b><font size="7" color="red">ch15. 데이터베이스 연동</font></b>
# 1절 SQLite 데이터 베이스 연결
- SQLite 데이터 베이스는 별도의 DBMS 없이 SQL문 이용해서 DB액세스할 수 있도록 만든 간단한 디스크 기반 DB
- C라이브러리
- SQLite는 프로토타입 생성시 사용
- 프로젝트 : 분석   ->   설계  ->  구현  -> 테스트 -> 고객에게 배포 -> 유지보수
                프로토타입(SQLite)       완제품(Oracle, MySQL, Maria, ..)
- [DB browser for SQLite](https://sqlitebrowser.org/dl/)
## 1.1 SQLite browser 설치 및 sqlite 패키지 load